In [ ]:
!pip install pandas
!pip install numpy
!pip install scipy
!pip install sklearn

In [7]:
import pandas as pd
import numpy as np
import scipy as sp
import gc
import sklearn

In [8]:
classifcols = pd.read_pickle('../data/classifcols.pkl')
regrcols = pd.read_pickle('../data/regrcols.pkl')
labencdist = pd.read_pickle('../data/labencdist.pkl')
labencsector = pd.read_pickle('../data/labencsector.pkl')
onehotenc = pd.read_pickle('../data/onehotenc.pkl')

In [73]:
totcols = list(set(classifcols).union(set(regrcols)).union(set(['Div_Paid?'])))
liabcols = ['Short-term debt','Accounts payable', 'Taxes payable',
            'Accrued liabilities','Other current liabilities', 
            'Total current liabilities','Long-term debt','Average Payables',
            'Deferred taxes liabilities','Other long-term liabilities', 
            'Total non-current liabilities','Total liabilities',
            'Days Payables Outstanding']
assetcols = ['Total cash','Receivables','Inventories','Prepaid expenses', 
             'Other current assets','Total current assets','Goodwill',
             'Gross property, plant and equipment','Other long-term assets',
             'Net property, plant and equipment','Intangible assets', 
             'Total non-current assets','Total assets','Average Inventory',
             'Average Receivables',"Total liabilities and stockholders' equity",
             'Cash per Share','Days Sales Outstanding','Days of Inventory on Hand']
inccols = ['Gross profit','Research and development','Operating income', 
           'Sales, General and administrative','Interest Expense',
           'Total operating expenses', 'Other income (expense)', 
           'Income before taxes','Provision for income taxes', 'Net income', 
           'Basic_EPS','Diluted_EPS', 'EBITDA'] 
equitycols = ['Common stock', 'Additional paid-in capital','Retained earnings', 
               'Treasury stock','Accumulated other comprehensive income']

def mdlinpt(comp):
    alldata = pd.read_pickle('../data/sp500finaldata.pkl')
    alldata.sort_values(by=['firm', 'Year'], ascending=[True, True], inplace=True)
    distenc = pd.DataFrame(labencdist.transform(alldata['DistanceFromLast']))
    sectenc = pd.DataFrame(labencsector.transform(alldata['Sector']))
    hotencoded = onehotenc.transform(pd.concat([sectenc, distenc], axis=1))
    namesdist = ['DistLast_'+i for i in labencdist.classes_]
    namessect = ['Sector_'+i for i in labencsector.classes_]
    names = np.append(namessect, namesdist)
    hotencoded = pd.DataFrame(hotencoded, columns=names)
    alldata.drop(['Sector', 'DistanceFromLast'], axis=1, inplace=True)
    alldata = pd.concat([alldata, hotencoded], axis=1)
    maxrec = alldata[(alldata.firm==comp) & (alldata.firm != alldata.firm.shift(-1))]
    maxrec = maxrec[totcols].reset_index(drop=True)
    return maxrec

def updater(maxrec, grrev, grinc, grasset, grliab, grcapex):
    maxrec[assetcols] *= (1+ grasset/100)
    maxrec[liabcols] *= (1 + grliab/100)
    maxrec[inccols] *= (1 + grinc/100)
    maxrec['Revenue'] *= (1 + grrev/100)
    maxrec['Cost of revenue'] = maxrec['Revenue'] - maxrec['Gross profit']
    oldeq = maxrec["Total stockholders' equity"]
    maxrec["Total stockholders' equity"] = maxrec['Total assets'] - maxrec['Total liabilities']
    oldeq = maxrec["Total stockholders' equity"] / oldeq
    maxrec[equitycols] *= oldeq
    #Updated final ratios
    maxrec['Intangibles to Total Assets']  = maxrec['Intangible assets'] / maxrec['Total assets']
    
    return maxrec

In [72]:
set(totcols) - set(liabcols) - set(assetcols) - set(equitycols) - set(inccols)

{'Book Value per Share',
 'Capex to Revenue',
 'Cost of revenue',
 'Debt to Assets',
 'DistLast_1',
 'DistLast_3',
 'DistLast_4',
 'DistLast_5',
 'DistLast_6',
 'Div_Paid?',
 'EV to Free cash flow',
 'EV to Sales',
 'Earnings Yield',
 'Enterprise Value',
 'Enterprise Value over EBITDA',
 'Free Cash Flow Yield',
 'Free Cash Flow per Share',
 'GDP',
 'GNI',
 'Graham Net-Net',
 'Graham Number',
 'Income Quality',
 'Index Value',
 'Intangibles to Total Assets',
 'Interest Coverage',
 'Interest Debt per Share',
 'Inventory Turnover',
 'Invested Capital',
 'Market Cap',
 'Market_Cap',
 'Net Current Asset Value',
 'Net Debt to EBITDA',
 'Net Income per Share',
 'Operating Cash Flow per Share',
 'PB ratio',
 'PBratio',
 'PE ratio',
 'PFCF ratio',
 'PTB ratio',
 'Paid_LastYr?',
 'Payables Turnover',
 'Price to Sales Ratio',
 'PropSectorAvgMktCap',
 'R&D to Revenue',
 'Revenue',
 'Revenue per Share',
 'SG&A to Revenue',
 'SectPaidLastYr',
 'Sector_Real Estate',
 'Shareholders Equity per Share',


In [52]:
set() - set(totcols)

set()

In [74]:
updater(mdlinpt('MSFT'),1,2,3,4,5)

KeyError: ''